In [1]:
import pickle

import keras
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
import pandas as pd

In [10]:
base_dir = "dataset"

ir = 3 # interpolate interval
before = 2
after = 2

# normalize
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

In [11]:
from scipy.interpolate import CubicSpline
def interpolate_numpy_array(arr, desired_length):
    cs = CubicSpline(np.linspace(0, 1, len(arr)), arr)
    x_new = np.linspace(0, 1, desired_length)
    interpolated_arr = cs(x_new)
    return interpolated_arr

In [12]:
def load_data():
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)

    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]
    groups_train = apnea_ecg["groups_train"]
    for i in range(len(o_train)):
        min_distance_list, max_distance_list, standard_deviation_distance_list = o_train[i]
		# Curve interpolation
        min_distance_list_inter = interpolate_numpy_array(min_distance_list,900)
        max_distance_list_inter = interpolate_numpy_array(max_distance_list,900)
        x_train.append([min_distance_list_inter, max_distance_list_inter])
    x_train = np.array(x_train, dtype="float32").transpose((0, 2, 1)) # convert to numpy format
    y_train = np.array(y_train, dtype="float32")

    x_test = []
    o_test, y_test = apnea_ecg["o_test"], apnea_ecg["y_test"]
    groups_test = apnea_ecg["groups_test"]
    for i in range(len(o_test)):
        min_distance_list, max_distance_list, standard_deviation_distance_list = o_test[i]
		# Curve interpolation
        min_distance_list_inter = interpolate_numpy_array(min_distance_list,900)
        max_distance_list_inter = interpolate_numpy_array(max_distance_list,900)
        x_test.append([min_distance_list_inter, max_distance_list_inter])
    x_test = np.array(x_test, dtype="float32").transpose((0, 2, 1))
    y_test = np.array(y_test, dtype="float32")

    return x_train, y_train, groups_train, x_test, y_test, groups_test

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten
from tensorflow.keras.layers import Input, Add, Activation, BatchNormalization
from tensorflow.keras.models import Model

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def bottleneck_block(inputs, filters, strides=1):
    # First Conv1D with 1x1 kernel (reduce dimensionality)
    x = layers.Conv1D(filters[0], kernel_size=1, strides=strides)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second Conv1D with 3x3 kernel (main convolution)
    x = layers.Conv1D(filters[1], kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Third Conv1D with 1x1 kernel (increase dimensionality)
    x = layers.Conv1D(filters[2], kernel_size=1)(x)
    x = layers.BatchNormalization()(x)

    # Identity skip connection if strides == 1 and filters[0] == filters[2]
    if strides == 1 and filters[0] == filters[2]:
        x = layers.add([x, inputs])

    x = layers.Activation('relu')(x)
    return x

def build_resnet50(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Initial Convolutional Layer
    x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)

    # ResNet Blocks
    block_sizes = [3, 4, 6, 3]  # Specifies the number of blocks in each set
    filters_list = [[64, 64, 256], [128, 128, 512], [256, 256, 1024], [512, 512, 2048]]  # Filters for each block

    for i, block_size in enumerate(block_sizes):
        for j in range(block_size):
            strides = 2 if i > 0 and j == 0 else 1
            x = bottleneck_block(x, filters_list[i], strides=strides)

    # Global Average Pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs, outputs, name='resnet50')
    return model

In [15]:
def lr_schedule(epoch, lr):
    if epoch > 70 and \
            (epoch - 1) % 10 == 0:
        lr *= 0.1
    print("Learning rate: ", lr)
    return lr

In [16]:
def plot(history):
    """Plot performance curve"""
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    axes[0].plot(history["loss"], "r-", history["val_loss"], "b-", linewidth=0.5)
    axes[0].set_title("Loss")
    axes[1].plot(history["accuracy"], "r-", history["val_accuracy"], "b-", linewidth=0.5)
    axes[1].set_title("Accuracy")
    fig.tight_layout()
    plt.show()

In [23]:
if __name__ == "__main__":
    x_train, y_train, groups_train, x_test, y_test, groups_test = load_data()

    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

    print("train num:", len(y_train))
    print("test num:", len(y_test))

    input_shape = (900, 2)
    num_classes = 2
    model = build_resnet50(input_shape, num_classes)
    model.summary()

    tf.keras.utils.plot_model(model, "model.png", show_shapes=True) # Plot model

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule) # Dynamic adjustment learning rate

    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test),
                        callbacks=[lr_scheduler])

    model.save(os.path.join("models", "model.final1.h5")) # Save training model

    loss, accuracy = model.evaluate(x_test, y_test) # test the model
    print("Test loss: ", loss)
    print("Accuracy: ", accuracy)


train num: 16713
test num: 16946
Model: "resnet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 900, 2)]          0         
                                                                 
 conv1d_50 (Conv1D)          (None, 450, 64)           960       
                                                                 
 batch_normalization_50 (Ba  (None, 450, 64)           256       
 tchNormalization)                                               
                                                                 
 activation_50 (Activation)  (None, 450, 64)           0         
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 225, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_51 (Conv1D)      

                                                                 
 activation_66 (Activation)  (None, 113, 128)          0         
                                                                 
 conv1d_67 (Conv1D)          (None, 113, 128)          49280     
                                                                 
 batch_normalization_67 (Ba  (None, 113, 128)          512       
 tchNormalization)                                               
                                                                 
 activation_67 (Activation)  (None, 113, 128)          0         
                                                                 
 conv1d_68 (Conv1D)          (None, 113, 512)          66048     
                                                                 
 batch_normalization_68 (Ba  (None, 113, 512)          2048      
 tchNormalization)                                               
                                                                 
 activatio

 tchNormalization)                                               
                                                                 
 activation_84 (Activation)  (None, 57, 256)           0         
                                                                 
 conv1d_85 (Conv1D)          (None, 57, 256)           196864    
                                                                 
 batch_normalization_85 (Ba  (None, 57, 256)           1024      
 tchNormalization)                                               
                                                                 
 activation_85 (Activation)  (None, 57, 256)           0         
                                                                 
 conv1d_86 (Conv1D)          (None, 57, 1024)          263168    
                                                                 
 batch_normalization_86 (Ba  (None, 57, 1024)          4096      
 tchNormalization)                                               
          

KeyboardInterrupt: 

In [28]:
#export to csv
y_score = model.predict(x_test)
output = pd.DataFrame({"y_true": y_test[:, 1], "y_score": y_score[:, 1], "subject": groups_test})
output.to_csv(os.path.join("output", "RESNEXT50.csv"), index=False)